In [1]:
# Relative Grading: Outputs A or B

from prometheus_eval import PrometheusEval
from prometheus_eval.prompts import RELATIVE_PROMPT

judge = PrometheusEval(model_id="prometheus-eval/prometheus-7b-v2.0", relative_grade_template=RELATIVE_PROMPT)


/data/home/kirteemishra/anaconda3/envs/baseclone/lib/python3.11/site-packages/prometheus_eval/judge.py:36: UserWarning: Reference answer was not given in Absolute Grading mode. This might lead to nonoptimal performances.
  warnings.warn(
/data/home/kirteemishra/anaconda3/envs/baseclone/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


INFO 05-25 11:34:10 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='prometheus-eval/prometheus-7b-v2.0', speculative_config=None, tokenizer='prometheus-eval/prometheus-7b-v2.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=prometheus-eval/prometheus-7b-v2.0)
INFO 05-25 11:34:11 utils.py:660] Found nccl from library /data/home/kirteemishra/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-25 11:34:13 selector.py:81] Cannot use FlashAttention-2 backend because the flash_attn package is not found. Please install it for better performance.
INFO 05-25 11:

In [2]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# device = "cuda" # the device to load the model onto

# model = AutoModelForCausalLM.from_pretrained("prometheus-eval/prometheus-7b-v2.0")
# tokenizer = AutoTokenizer.from_pretrained("prometheus-eval/prometheus-7b-v2.0")

# ABS_SYSTEM_PROMPT = "You are a fair judge assistant tasked with providing clear, objective feedback based on specific criteria, ensuring each assessment reflects the absolute standards set for performance."

# ABSOLUTE_PROMPT = """###Task Description:
# An instruction (might include an Input inside it), a response to evaluate, a reference answer that gets a score of 5, and a score rubric representing a evaluation criteria are given.
# 1. Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
# 2. After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.
# 3. The output format should look as follows: "Feedback: (write a feedback for criteria) [RESULT] (an integer number between 1 and 5)"
# 4. Please do not generate any other opening, closing, and explanations.

# ###The instruction to evaluate:
# {instruction}

# ###Response to evaluate:
# {response}

# ###Reference Answer (Score 5):
# {reference_answer}

# ###Score Rubrics:
# {rubric}

# ###Feedback: """

# user_content = ABS_SYSTEM_PROMPT + "\n\n" + ABSOLUTE_PROMPT.format("a", "b", ) # Fill the prompt with your data

# messages = [
#     {"role": "user", "content": user_content},
# ]

# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

# model_inputs = encodeds.to(device)
# model.to(device)

# generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
# decoded = tokenizer.batch_decode(generated_ids)
# print(decoded[0])



In [3]:
import pandas as pd

ans_df = pd.read_csv("/data/home/kirteemishra/sahil/llms/llama3/data/judge_exps/ans_test.csv")

In [4]:
ft_inc_ans = ans_df['ft_inc'].tolist()
base_inc_ans = ans_df['base'].tolist()


In [5]:
ft_inc_ans

['To increase Ragi (Jowar) yield, ensure:\n\n1. Adequate moisture: Ragi requires well-distributed rainfall or irrigation.\n2. Balanced fertilizer: Apply 15:15:15 NPK at sowing time, followed by 10:10:10 NPK at 30 days after sowing.\n3. Adequate spacing: Maintain 15-20 cm row-to-row and 10-15 cm plant-to-plant spacing.\n4. Weed control: Regular',
 'For control of sucking pests in mulberry, spray Dimethoate 20 EC at 2ml/litre of water.',
 'For controlling grass weeds, I recommend applying 2,4-D at 1.5 kg/ha as a pre-emergence treatment or fluazifop at 0.3 kg/ha as a post-emergence treatment.',
 'Spray carbendazim 0.1% or carbendazim + triadimefon 0.1% + 0.25% at 30-40 days after sowing.',
 'For powdery mildew control in Croton, apply 0.1% Bordeaux mixture or 0.1% Copper oxychloride spray at 15 days interval.',
 '\n\nFor caterpillar control in rose, use Bt-based pesticides or neem oil. For leaf spot, use copper-based fungicides or Bordeaux mixture.',
 "Dwarf varieties of coconut suitable 

In [6]:
ft_inc_ans[0]

'To increase Ragi (Jowar) yield, ensure:\n\n1. Adequate moisture: Ragi requires well-distributed rainfall or irrigation.\n2. Balanced fertilizer: Apply 15:15:15 NPK at sowing time, followed by 10:10:10 NPK at 30 days after sowing.\n3. Adequate spacing: Maintain 15-20 cm row-to-row and 10-15 cm plant-to-plant spacing.\n4. Weed control: Regular'

In [7]:
query = [f"A farmer from Karnataka asked {x}" for x in ans_df['Query']]
kcc_ans = ans_df['KccAns'].tolist()

In [8]:
rubric = '''Pease act as an impartial judge and evaluate the quality of the responses provided by two
            AI assistants to the user question displayed below. Your evaluation should consider
            correctness and helpfulness. You will be given a reference answer, assistant A’s answer,
            and assistant B’s answer. Your job is to evaluate which assistant’s answer is better.
            Begin your evaluation by comparing both assistants’ answers with the reference answer.
            Identify and correct any mistakes. Avoid any position biases and ensure that the order in
            which the responses were presented does not influence your decision. Do not allow the
            length of the responses to influence your evaluation.'''

In [9]:

feedbacks, scores = judge.relative_grade(
    instructions=query,
    responses_A=ft_inc_ans,
    responses_B=base_inc_ans,
    params={},
    rubric=rubric,
    reference_answers=kcc_ans
)

Processed prompts: 100%|██████████| 8/8 [00:09<00:00,  1.24s/it]


Processed 8/8 instances.


Finalizing: 100%|██████████| 8/8 [00:00<00:00, 205855.41it/s]


In [10]:
scores

['A', 'B', 'B', 'B', 'B', 'B', 'A', 'B']

In [11]:
feedbacks

["Both Response A and Response B provide helpful recommendations for increasing the yield of Ragi. They both address key aspects such as adequate moisture, balanced fertilization, and weed control. However, there are some notable differences that make Response A stand out as better.\n\nResponse A specifies the type of fertilizer to be used (NPK) and the application rates (15:15:15 and 10:10:10), which directly aligns with the recommended agronomic practices for Ragi cultivation. It also emphasizes the importance of adequate spacing for the plants, which is a crucial factor for improving yield and preventing diseases. On the other hand, Response B mentions optimal soil pH, which is indeed important for plant growth, but it fails to mention the specific fertilizers and their rates. Additionally, it only provides a general recommendation for balanced fertilization without giving detailed instructions.\n\nFurthermore, while both responses address the necessity of adequate moisture, Respons

In [12]:
A = "ft_inc"
B = "base"
dir = f"{A}_vs_{B}"
pd.DataFrame({f'{dir}_scores': scores,f'{dir}_feedback': feedbacks}).to_csv(f"{dir}_prom.csv")